# Milestone 2: Group 17
Authors: Rus Dimitrov, Elena Ganacheva, Luke Yang, Shirley Zhang

> **Notes** 
> Preparing data for an ML model to predict rainfall over Sydney at a certain time 

> 1) Read the data from your s3 bucket in the cloud (use the pandas library, read_parquet(location_of_file, aws_credential))
>    - For now, keep it as a local file 
>    - Later, change the location 
> 2) Filter the data by location (Sydney) 
>    - Minimum and maximum latitude, longitude
>    - Keep it within Sydney latitute and longitude 
> 3) Remove redundant information 
>    - Remove redundant columns
> 4) Prepare the data for a model 
>    - Compare models against ground truth --> add a column that has the true rainfall values (y_truth), supervised ML problem 

In [1]:
# imports 
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
#import pyarrow.dataset as ds

## 1. Read the data 

TO CHANGE: 
- Update the file location to the aws s3 bucket
- Add credentials

In [2]:
# convert our .csv file to parquet (only needs to be run once, comment this out after)
# reference: https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture2.html?highlight=parquet#converting-csv-parquet

#########################
#%time 
#df = ds.dataset(file_csv, format="csv")
#result = df.scanner()
#ds.write_dataset(result, 
#                 file_parquet, 
#                 format = "parquet", 
#                 #partitioning=["year"],
#                 #partitioning_flavor="hive", 
#                 #existing_data_behavior = 'overwrite_or_ignore'
#                )
#########################
# read the parquet file as pandas 
#%time
#df = pd.read_parquet(file_parquet)
#########################
#df.shape # (62513863, 7)

In [3]:
# reference: https://github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/blob/master/release/milestone2/Milestone2.ipynb
# do the following once: 

In [4]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}

#output_directory = "/srv/data/my_shared_data_folder/"
output_directory = "/Users/shirley/Desktop/525_cloud/data/"

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26844650,
  'name': 'allyears.csv.zip',
  'size': 2405908113,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81'},
 {'id': 26863682,
  'name': 'individual_years.zip',
  'size': 1896206676,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77'},
 {'id': 27515426,
  'name': 'combined_model_data.csv.zip',
  'size': 821308997,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d'},
 {'id': 27520682,
  'name': 'combined_model_data_parti.parquet.zip',
  'size': 519743915,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/

In [6]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

KeyboardInterrupt: 

In [ ]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

In [2]:
# set up file locations

# 1) combined_model_data_parti.parquet
combined_parquet = '/Users/shirley/Desktop/525_cloud/data/combined_model_data_parti.parquet'
# 2) observed_daily_rainfall_SYD.csv
observed_SYD_csv = '/Users/shirley/Desktop/525_cloud/data/observed_daily_rainfall_SYD.csv'

In [3]:
# read the parquet file as pandas 
%time
combined_df = pd.read_parquet(combined_parquet)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [4]:
combined_df.shape

(62513863, 7)

In [5]:
combined_df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [6]:
# set time to index 
combined_df = combined_df.set_index('time')

In [7]:
combined_df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [37]:
# read the observed SYD data 
%time 
observed_SYD_df = pd.read_csv(observed_SYD_csv, index_col='time', parse_dates=['time'])

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 11.9 µs


In [38]:
observed_SYD_df.shape

(46020, 1)

In [39]:
observed_SYD_df.head()

,rain (mm/day)
time,
1889-01-01,0.006612
1889-01-02,0.090422
1889-01-03,1.401452
1889-01-04,14.869798
1889-01-05,0.467628


In [40]:
observed_SYD_df.index

DatetimeIndex(['1889-01-01', '1889-01-02', '1889-01-03', '1889-01-04',
               '1889-01-05', '1889-01-06', '1889-01-07', '1889-01-08',
               '1889-01-09', '1889-01-10',
               ...
               '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
               '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
               '2014-12-30', '2014-12-31'],
              dtype='datetime64[ns]', name='time', length=46020, freq=None)

## 2. Filter the data by location

In [11]:
# define latitude and longitude of Sydney 
syd_lat = -33.86
syd_lon = 151.21

In [23]:
# filter combined data 
syd_query = 'lat_min <= @syd_lat & lat_max >= @syd_lat & lon_min <= @syd_lon & lon_max >= @syd_lon'
filtered_combined_df = combined_df.query(syd_query) 

In [24]:
filtered_combined_df.shape

(1150049, 6)

In [25]:
filtered_combined_df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.0,-33.75,150.0,151.875,0.040427,ACCESS-CM2
1889-01-02 12:00:00,-35.0,-33.75,150.0,151.875,0.073777,ACCESS-CM2
1889-01-03 12:00:00,-35.0,-33.75,150.0,151.875,0.232656,ACCESS-CM2
1889-01-04 12:00:00,-35.0,-33.75,150.0,151.875,0.911319,ACCESS-CM2
1889-01-05 12:00:00,-35.0,-33.75,150.0,151.875,0.698013,ACCESS-CM2


## 3. Remove redundant information

In [26]:
# drop latitude and longitude columns 
drop_combined_df = filtered_combined_df.drop(['lat_min', 'lat_max', 'lon_min', 'lon_max'], axis=1)

In [27]:
drop_combined_df.head()

,rain (mm/day),model
time,,
1889-01-01 12:00:00,0.040427,ACCESS-CM2
1889-01-02 12:00:00,0.073777,ACCESS-CM2
1889-01-03 12:00:00,0.232656,ACCESS-CM2
1889-01-04 12:00:00,0.911319,ACCESS-CM2
1889-01-05 12:00:00,0.698013,ACCESS-CM2


In [28]:
drop_combined_df.shape

(1150049, 2)

## 4. Prepare the data for a model

In [29]:
# pivot the dataframe
pivot_combined_df = drop_combined_df.pivot(columns='model', values='rain (mm/day)')

In [30]:
pivot_combined_df.head()

model,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1889-01-01 12:00:00,0.040427,1.814552,35.579336,4.268112,0.001107,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,0.648031,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,15.760960,0.000048,2.451075,0.221324,2.257933
1889-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1889-01-02 12:00:00,0.073777,0.303965,4.596520,1.190141,0.000102,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,1.099883,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,0.367551,0.435086,0.477231,3.757179,2.287381
1889-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
observed_SYD_df.head()

,rain (mm/day)
time,
1889-01-01,0.006612
1889-01-02,0.090422
1889-01-03,1.401452
1889-01-04,14.869798
1889-01-05,0.467628


In [54]:
# combine model and observed data 
#pd.merge(pivot_combined_df, observed_SYD_df, left_index=True, right_index=True)

In [56]:
pivot_combined_df.query('time == "1889-01-01 12:00:00"')

model,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01 12:00:00,0.040427,1.814552,35.579336,4.268112,0.001107,11.410537,3.322009e-08,2.6688,1.321215,1.515293,...,0.648031,4.244226e-13,1.390174e-13,0.000065,0.000003,15.76096,0.000048,2.451075,0.221324,2.257933


In [58]:
resampled_combined_df = pivot_combined_df.resample('1D').sum()

In [59]:
resampled_combined_df.shape

(46020, 25)

In [61]:
resampled_combined_df.columns

CategoricalIndex(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-ESM-1-1-LR',
                  'BCC-CSM2-MR', 'BCC-ESM1', 'CMCC-CM2-HR4', 'CMCC-CM2-SR5',
                  'CMCC-ESM2', 'CanESM5', 'EC-Earth3-Veg-LR', 'FGOALS-g3',
                  'GFDL-CM4', 'INM-CM4-8', 'INM-CM5-0', 'KIOST-ESM', 'MIROC6',
                  'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR',
                  'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM',
                  'SAM0-UNICON', 'TaiESM1'],
                 categories=['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', ..., 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1', 'observed'], ordered=False, dtype='category', name='model')

In [63]:
observed_SYD_df.head()

,rain (mm/day)
time,
1889-01-01,0.006612
1889-01-02,0.090422
1889-01-03,1.401452
1889-01-04,14.869798
1889-01-05,0.467628


In [64]:
merged_df = pd.merge(resampled_combined_df, observed_SYD_df, left_index=True, right_index=True)

In [67]:
# rename observed rainfall column 
final_df = merged_df.rename(columns={'rain (mm/day)': 'observed'})

In [68]:
final_df.head()

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,observed
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,11.920356,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628


## 5. Save the file 

TO CHANGE: 
- Update the file location to the aws s3 bucket

In [72]:
# output file location 
ml_data_SYD_csv = '/Users/shirley/Desktop/525_cloud/data/ml_data_SYD.csv'

In [73]:
# save to csv 
%time
final_df.to_csv(ml_data_SYD_csv)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 27.9 µs


In [78]:
# double check 
pd.read_csv('/Users/shirley/Desktop/525_cloud/data/ml_data_SYD.csv', index_col='time')

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,observed
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,3.557934e+01,4.268112e+00,1.107466e-03,1.141054e+01,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520e+00,1.190141e+00,1.015323e-04,4.014984e+00,1.312700e+00,0.946211,2.788724,4.771375,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467e+00,1.003845e-09,1.760345e-05,9.660565e+00,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624e+00,8.225225e-02,1.808932e-01,3.951528e+00,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686e+00,2.496841e+00,4.708019e-09,2.766362e+00,1.822940e+01,0.339267,0.002468,11.920356,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,0.033748,0.123476,1.451179e+00,3.852845e+01,2.061717e-03,8.179260e-09,1.171263e-02,0.090786,59.895053,5.071783,...,4.726998e-13,1.326889e-01,1.827857e+00,6.912632e-03,2.171327e-03,1.620489e+00,2.084252,0.868046,17.444923,0.037472
2014-12-28,0.094198,2.645496,4.249335e+01,5.833801e-01,5.939502e-09,8.146937e-01,4.938899e-01,0.000000,0.512632,1.578188,...,4.609420e-13,1.644482e+00,7.242920e-01,2.836752e-03,1.344768e+01,2.391159e+00,1.644527,0.782258,1.569647,0.158061
2014-12-29,0.005964,3.041667,2.898325e+00,9.359547e-02,2.000051e-08,2.532205e-01,1.306046e+00,0.000002,37.169669,1.565885,...,2.016156e+01,1.506439e+00,1.049481e-01,8.137182e+00,2.547820e+01,1.987695e-12,0.205036,2.140723,1.444630,0.025719
